In [1]:
import pandas as pd
import numpy as np
from sets import Set

/Users/yyc/env/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: the sets module is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
# visulize the yoochoose-clicks.dat dataset
clicks = pd.read_csv('yoochoose_data/yoochoose-clicks.dat', header=None, names=['Session ID','Timestamp','Item ID','Category'])
print clicks.iloc[:5, :]
print 'Total data length: ' + str(len(clicks.index))

/Users/yyc/env/tensorflow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Session ID                 Timestamp    Item ID Category
0           1  2014-04-07T10:51:09.277Z  214536502        0
1           1  2014-04-07T10:54:09.868Z  214536500        0
2           1  2014-04-07T10:54:46.998Z  214536506        0
3           1  2014-04-07T10:57:00.306Z  214577561        0
4           2  2014-04-07T13:56:37.614Z  214662742        0
Total data length: 33003944


In [3]:
# visulize the yoochoose-buys.dat dataset
buys = pd.read_csv('yoochoose_data/yoochoose-buys.dat', header=None, names=['Session ID','Timestamp','Item ID','Price','Quantity'])
print buys.iloc[:5, :]
print 'Total data length: ' + str(len(buys.index))

   Session ID                 Timestamp    Item ID  Price  Quantity
0      420374  2014-04-06T18:44:58.314Z  214537888  12462         1
1      420374  2014-04-06T18:44:58.325Z  214537850  10471         1
2      281626  2014-04-06T09:40:13.032Z  214535653   1883         1
3      420368  2014-04-04T06:13:28.848Z  214530572   6073         1
4      420368  2014-04-04T06:13:28.858Z  214835025   2617         1
Total data length: 1150753


In [4]:
# visulize the yoochoose-test.dat dataset
test = pd.read_csv('yoochoose_data/yoochoose-test.dat', header=None, names=['Session ID','Timestamp','Item ID','Category'])
print test.iloc[:5, :]
print 'Total data length: ' + str(len(test.index))

   Session ID                 Timestamp    Item ID Category
0           5  2014-04-07T17:13:46.713Z  214530776        0
1           5  2014-04-07T17:20:56.973Z  214530776        0
2           5  2014-04-07T17:21:19.602Z  214530776        0
3          10  2014-04-04T07:44:14.590Z  214820942        0
4          10  2014-04-04T07:45:20.245Z  214826810        0
Total data length: 8251791


In [4]:
# create a new dataframe with clicks frequency
clicks['Popularity'] = clicks.groupby('Item ID')['Item ID'].transform('count') 
clicks_freq = clicks[['Item ID','Popularity']].drop_duplicates(subset='Item ID', keep='first')
clicks_freq = clicks_freq.sort_values('Popularity', ascending=False)
print clicks_freq.iloc[:5, :]
print len(clicks_freq)

clicks['Sess cnt'] = clicks.groupby('Session ID')['Session ID'].transform('count')
clicks_sess = clicks[['Session ID','Sess cnt']].drop_duplicates(subset='Session ID', keep='first')
print clicks_sess.iloc[:5, :]
print len(clicks_sess)

            Item ID  Popularity
5643722   643078800      147419
21981217  214853094      125690
4961288   214829878      113488
24943883  214853420       78448
1607291   214826610       69710
52739
    Session ID  Sess cnt
0            4   3743564
4            6   2179104
10           3   4840068
13           2   7116174
24          12    631824
188


In [6]:
# create a new dataframe with purchase frequency
buys['Popularity'] = buys.groupby('Item ID')['Item ID'].transform('count') 
buys_freq = buys[['Item ID', 'Price','Quantity','Popularity']]
buys_freq = buys_freq.drop_duplicates(subset='Item ID', keep='first')
buys_freq = buys_freq[buys_freq['Price'] > 50]
buys_freq['Popularity'] = buys_freq['Popularity'] * buys_freq['Quantity']
buys_freq = buys_freq.sort_values('Popularity', ascending=False)
buys_freq = buys_freq[['Item ID', 'Popularity']]
print buys_freq.iloc[:5, :]

          Item ID  Popularity
40      214821277       14344
98753   214844394       14226
4308    214821302        9880
840925  214850947        8961
126738  214829861        7032


In [7]:
from sets import Set
# get the item which price < 50
set = Set()
cnt = 0
for row in buys.itertuples():
    if row[-3] < 50:
        cnt += 1;
        set.add(row[-4])
print cnt
print len(set)

610283
15162


In [8]:
msk = np.random.rand(len(test)) < 0.8
test_for_train = test[msk]
test_for_test = test[~msk]
print test_for_test.iloc[0:5, :]
print 'length of train:' + str(len(test_for_train))
print 'length of test:' + str(len(test_for_test))

    Session ID                 Timestamp    Item ID Category
6           15  2014-04-05T08:15:49.200Z  214547255        0
37          50  2014-04-01T11:15:02.408Z  214718169        0
38          50  2014-04-01T11:15:03.514Z  214821015        0
46          55  2014-04-07T19:31:05.145Z  214840775        0
49          70  2014-04-05T12:41:35.624Z  214827022        0
length of train:6601411
length of test:1650380


In [9]:
test['count'] = test.groupby('Session ID')['Session ID'].transform('count') 
test_count = test[['Session ID','Item ID','Category','count']] #drop_duplicates(subset='Session ID', keep='first')
print test_count.iloc[:5, :]

   Session ID    Item ID Category  count
0           5  214530776        0      3
1           5  214530776        0      3
2           5  214530776        0      3
3          10  214820942        0      2
4          10  214826810        0      2


In [20]:
items_list = clicks['Item ID'].drop_duplicates().sort_values().reset_index(drop=True)
print items_list

0         214507224
1         214507226
2         214507228
3         214507239
4         214507256
5         214507331
6         214507365
7         214507385
8         214507387
9         214507408
10        214507415
11        214507445
12        214507447
13        214507477
14        214507479
15        214507481
16        214507492
17        214507500
18        214507556
19        214507610
20        214507678
21        214507708
22        214507723
23        214507802
24        214507897
25        214507927
26        214507946
27        214507948
28        214507955
29        214507982
            ...    
52709     643078950
52710    1178793047
52711    1178794001
52712    1178794852
52713    1178796523
52714    1178796800
52715    1178799879
52716    1178804544
52717    1178812378
52718    1178818515
52719    1178818815
52720    1178819852
52721    1178820367
52722    1178820399
52723    1178824421
52724    1178825045
52725    1178826028
52726    1178826150
52727    1178827543
